In [94]:
import pandas as pd
import numpy as np 
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Dataset, Data
from torch_geometric.transforms import NormalizeFeatures
from torch.nn import Linear
from torchmetrics import ConfusionMatrix, AUROC, F1Score, Precision, Recall

import os
from tqdm import tqdm

In [95]:
MIN_DISEASE_S_GENE_NUMBER = 0
TEST_TRAIN_SPLIT = 0.5
TEST_VAL_SPLIT = 0.5
torch.manual_seed(42)

In [96]:
class IdMapper():
    sorted_diseases = []
    sorted_genes = []

    def __init__(self, gene_file, disease_file):
        genes = pd.read_csv(gene_file, sep="\t")
        self.genes = genes["genes"].sort_values().unique()

        disieses = pd.read_csv(disease_file, sep="\t")
        diseases_filtered = disieses.groupby("diseaseId").filter(lambda x: len(x) > MIN_DISEASE_S_GENE_NUMBER)
        self.diseases = diseases_filtered["diseaseId"].sort_values().unique()

    def diseases_idx_to_id_map(self):
        return { idx: item  for idx, item in enumerate(self.diseases)}
    
    def diseases_id_to_idx_map(self):
        return { item: idx  for idx, item in enumerate(self.diseases)}
    
    def genes_idx_to_id_map(self):
        return { idx: item  for idx, item in enumerate(self.genes)}
   
    def genes_id_to_idx_map(self):
        return { item: idx  for idx, item in enumerate(self.genes)}

In [ ]:
class GeneDataset(Dataset):
    def __init__(self, root, filenames, test_size, val_size, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.test_size = test_size
        self.val_size = val_size
        self.filenames = filenames
        self.mapper = IdMapper("../data/raw/"+filenames[0], "../data/raw/"+filenames[2])
        super(GeneDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filenames

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        if self.test:
            return [F'{file_name}_test' for file_name in self.raw_paths]
        else:
            return self.raw_paths

    def download(self):
        pass

    def process(self):
        self.genes_features = pd.read_csv(self.raw_paths[0], sep="\t")
        self.edges_features = pd.read_csv(self.raw_paths[1], sep="\t")
        self.disiese_gene_matrix = pd.read_csv(self.raw_paths[2], sep="\t")

        self.genes = self.genes_features["genes"].sort_values().unique()
        self.diseases = self.disiese_gene_matrix["diseaseId"].sort_values().unique()

        node_feats = self._get_node_features(self.genes_features)
        edge_feats = self._get_edge_features(self.edges_features)
        edge_index = self._get_adjacency_info(self.edges_features)

        y = self._create_mask_matrix(self.disiese_gene_matrix.copy()).to(torch.float32)
        train_mask, validation_mask, test_mask = self._get_train_val_test_mask(self.disiese_gene_matrix.copy()) 

        data = Data(x=node_feats,
                    edge_index=edge_index,
                    edge_weight=edge_feats,
                    test_mask=test_mask, val_mask=validation_mask, train_mask=train_mask, y=y)
         
        if self.test:
            torch.save(data, os.path.join(self.processed_dir, 'graph_test.pt'))
        else:
            torch.save(data, os.path.join(self.processed_dir, 'graph.pt'))


    def _get_train_val_test_mask(self, disiese_gene_matrix):
        """ 
        i need too create matrices shape like disgenet
        and in this matrix i pick random points which are gonna be the train mask, validation mask and test mask
        
        in the train dataset i need to pick 80% from disgenet, equaly 0s and 1s in a column
        in the validation dataset i need to pick 10% from disgenet, equaly 0s and 1s in a column
        """

        train, validation, test = self._split_labels_to_train_val_test(disiese_gene_matrix)
        disgenet_inverse = self._get_disgenet_inverse(disiese_gene_matrix)
        train_n, validation_n, test_n = self._split_labels_to_train_val_test(disgenet_inverse)
        train_r = pd.concat([train, train_n], ignore_index=True)
        validation_r = pd.concat([validation, validation_n], ignore_index=True)
        test_r = pd.concat([test, test_n], ignore_index=True)

        train_mask = self._create_mask_matrix(train_r)
        validation_mask = self._create_mask_matrix(validation_r)
        test_mask = self._create_mask_matrix(test_r)

        return train_mask, validation_mask, test_mask
    
    def _split_labels_to_train_val_test(self, disgenet: pd.DataFrame):
        #Split the positive targets to equal partitions by disease
        disgenet_grouped = disgenet.groupby(by="diseaseId", group_keys=False)
        test_validation = disgenet_grouped.apply(lambda x: x.sample(frac=TEST_TRAIN_SPLIT, random_state=1))
        train = disgenet.drop(test_validation.index)
        test_validation_grouped = test_validation.groupby(by="diseaseId", group_keys=False)

        #Group by is needed before sample function call!!!
        test = test_validation_grouped.apply(lambda x: x.sample(frac=TEST_VAL_SPLIT, random_state=1))
        drop_indices = pd.concat([train, test]).index
        validation = disgenet.drop(drop_indices)
        return train, validation, test
    
    
    def _get_disgenet_inverse(self, disgenet):
        genes_frame = pd.DataFrame(list(self.genes), columns=["geneId"])
        diseases_frame = pd.DataFrame(self.diseases, columns=["diseaseId"])
        gene_disease_descartes_product = genes_frame.merge(diseases_frame, how="cross")
        disgenet_inverse = gene_disease_descartes_product.merge(disgenet, on=['geneId', 'diseaseId'], how='left', indicator=True)
        return disgenet_inverse[disgenet_inverse['_merge'] == 'left_only'].drop(columns='_merge')


    def _create_mask_matrix(self, dataframe):
        dataframe_for_matrix = pd.DataFrame(np.zeros((len(self.genes), len(self.diseases)),))
        dataframe_for_matrix = dataframe_for_matrix.astype(bool)
        dataframe_for_matrix[:] = False
        gene_id_to_idx = self.mapper.genes_id_to_idx_map()
        disease_id_to_idx = self.mapper.diseases_id_to_idx_map()
        
        dataframe["geneId"] = dataframe["geneId"].map(gene_id_to_idx) 
        dataframe["diseaseId"] = dataframe["diseaseId"].map(disease_id_to_idx)
        tuples_array = [row for row in dataframe.itertuples(index=False, name=None)]
        for row, col in tqdm(tuples_array):
            dataframe_for_matrix.loc[row, col] = True
        
        return torch.tensor(dataframe_for_matrix.to_numpy(), dtype=torch.bool)

    def _get_node_features(self, genes):
        gene_id_to_idx = self.mapper.genes_id_to_idx_map()
        genes["genes"] = self.genes_features["genes"].map(gene_id_to_idx)
        all_node_feats = genes.values.tolist()
        all_node_feats = np.asarray(all_node_feats)
        
        return torch.tensor(all_node_feats, dtype=torch.float32)

    def _get_edge_features(self, edges):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        duplicated_edges = edges.loc[edges.index.repeat(2)].reset_index(drop=True)
        all_edge_feats = duplicated_edges["combined_score"].tolist()
        return torch.tensor(all_edge_feats, dtype=torch.float32)


    def _get_adjacency_info(self, edges):
        """
        We want to be sure that the order of the indices
        matches the order of the edge features
        """
        gene_id_to_idx = self.mapper.genes_id_to_idx_map()

        edge_indices = []
        gene_1 = edges["gene1"].map(gene_id_to_idx)
        gene_2 = edges["gene2"].map(gene_id_to_idx)
        edges = pd.concat([gene_1, gene_2], axis=1).values.tolist()

        #iterate over the edges end duplicate it because for one edge we need: n1,n2 and n2,n1
        double_edges = []
        for edge in edges:
            double_edges += [ edge, [edge[1], edge[0]]]

        edge_indices = torch.tensor(double_edges)
        edge_indices = edge_indices.t().to(torch.int32).view(2, -1)
        return edge_indices

    def len(self):
        return self.genes.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            graph = torch.load(os.path.join(self.processed_dir, 'graph_test.pt'), weights_only=False)
        else:
            graph = torch.load(os.path.join(self.processed_dir, 'graph.pt'), weights_only=False)

        return graph
    
    def __getitem__(self, idx):
        return self.get(0)

In [98]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [99]:
dataset = GeneDataset(
    root="./data", 
    filenames=["gtex_genes.csv", "gene_graph.csv", "disgenet_with_gene_id.csv"],
    test_size=0.2,
    val_size=0.0,
    transform=NormalizeFeatures())

Processing...
100%|██████████| 60446/60446 [00:02<00:00, 25938.25it/s]
C:\Users\Nemes\AppData\Local\Temp\ipykernel_16728\1832578911.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_validation = disgenet_grouped.apply(lambda x: x.sample(frac=TEST_TRAIN_SPLIT, random_state=1))
C:\Users\Nemes\AppData\Local\Temp\ipykernel_16728\1832578911.py:88: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test = test

A gráf kirajzoltatása
(nagyon lassan fut le!!!!!!!!! --> 1 óra volt a colab-ban)

In [100]:
# import networkx as nx
# from torch_geometric.utils import to_networkx
# import matplotlib.pyplot as plt

# G = to_networkx(dataset[0], to_undirected=True)
# plt.figure(figsize=(100, 100))
# nx.draw(G, with_labels=False, node_color='lightblue', font_weight='bold')
# plt.savefig("graph.svg", format="svg")

disgenetet úgy tovább szűrni, hogy az egyes betegséghez legalább x gén tartozzon --> végén majd kiprobálni, hogy nem szürök rajtuk

GCN --> a veszteség függvény legyen jó, sima bináris osztályozás

keresztvalidáció

mátrixokkal dolgozzak

ha kiegyensulyozatlan akkor --> f1 score, avg precision, precision-recall görbe, (olyan metrikákat használjak)
                                    dúsitást NEEEE


In [ ]:
# The simple GCN modell
class GCN(torch.nn.Module):
    def __init__(self, dataset, hidden_channels):
        super().__init__()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.conv1 = GCNConv(dataset.x.shape[1], hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.out = Linear(hidden_channels, dataset.y.shape[1]) # dimension of disieses

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.conv2(x, edge_index, edge_weight)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        
        out = F.sigmoid(self.out(x))
        return out

In [102]:
data = dataset[0].to(device)
model = GCN(data, hidden_channels=16).to(device=device)

In [103]:
weight_decay = 0.0001
learning_rate = 0.01
sgd_momentum = 0.8
pos_weight = torch.tensor([1.0], dtype=torch.float32).to(device)

loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=learning_rate,
                            momentum=sgd_momentum,
                            weight_decay=weight_decay)

In [107]:
def train():
      model.train()
      optimizer.zero_grad() 

      # Use all data as input, because all nodes have node features
      out = model(data.x, data.edge_index, data.edge_weight)
      out_flaged = out.data.gt(1e-10).int()
      # Only use nodes with labels available for loss calculation --> mask
      print(out.shape, data.train_mask.shape)
      train_data_masked = torch.masked_select(out, data.train_mask)
      train_labels_masked = torch.masked_select(data.y, data.train_mask)
      loss = loss_fn( train_data_masked, train_labels_masked)  
      loss.backward() 
      optimizer.step()
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index, data.edge_weight)  
      # Check against ground-truth labels.
      out_flaged = out.data.gt(1e-10).int()
      test_data_masked = torch.masked_select(out_flaged, data.test_mask)
      test_labels_masked = torch.masked_select(data.y, data.test_mask)
      test_correct = torch.eq(test_data_masked, test_labels_masked)
      test_acc = int(test_correct.sum()) / int(len(test_correct))  
      return test_acc

def validation():
     model.eval()
     out = model(data.x, data.edge_index, data.edge_weight)
     return False

I need to create a map for idx - geneid, idx - diseaseid

In [ ]:
mapper = IdMapper("../data/raw/gtex_genes.csv", "../data/raw/disgenet_with_gene_id.csv")
gene_mapps = mapper.genes_idx_to_id_map()

In [108]:
best_loss = 1000
early_stopping_counter = 0
validation_stop = False

print(f"Model is on cuda: {next(model.parameters()).is_cuda}")
print(f"Data is on cuda: {data.is_cuda}")

for epoch in range(1, 2):
    if early_stopping_counter <= 20 or validation_stop:
        loss = train()
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

        if best_loss > loss:
            best_loss = loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        validation_stop = validation()
        if validation_stop:
            print("Early stopping due to overfitting.")
    else:
        print("Early stopping due to no improvement.")
        break;

print(f"Best loss: {best_loss}" )

Model is on cuda: False
Data is on cuda: False
torch.Size([15811, 1553]) torch.Size([15793, 1553])


RuntimeError: The size of tensor a (15793) must match the size of tensor b (15811) at non-singleton dimension 0

In [ ]:
test()

1.0